# Goal: Understand

Learnings
- inputs are [pos, neg] major. So my context extension code was wrong.
- Model is indeed causal


In [44]:
from einops import rearrange, repeat

rearrange(torch.tensor([1, 1, 1, 1, 2, 2, 2, 2]), '(a b)  -> a b', a=2, b=4)  # [1, 2, 3, 4]



tensor([[1, 1, 1, 1],
        [2, 2, 2, 2]])

In [1]:
import torch
import os
import sys
import surgery


sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), "ComfyUI", "custom_nodes"))

vtm = torch.load('vanilla_temporal_module_self.pt')
encoder_hidden_states = torch.load('vanilla_temporal_module_encoder_hidden_states.pt')
attention_mask = torch.load('vanilla_temporal_module_attention_mask.pt')

input_tensor = torch.load('vanilla_temporal_module_input_tensor.pt')
print(input_tensor.shape)

def get_frame_grads(output_index, force_causal):
    surgery.debug_options.set_from_dict({
        "force_causal": force_causal,
    })
    input_tensor = torch.load('vanilla_temporal_module_input_tensor.pt')
    input_tensor.requires_grad_()
    output_tensor = vtm.forward(input_tensor, encoder_hidden_states, attention_mask)
    output_tensor[output_index].max().backward()
    return [ (i, frame_grad.max().item()) for i, frame_grad in enumerate(input_tensor.grad)]


# gradient should be nonzero for future frames (i > 8) bc non-causal
get_frame_grads(8, False)



Total VRAM 81229 MB, total RAM 1814232 MB
Set vram state to: NORMAL_VRAM
Device: cuda:0 NVIDIA A100-SXM4-80GB : native
VAE dtype: torch.bfloat16
Using pytorch cross attention


/home/ubuntu/inf/ComfyUI/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([32, 320, 64, 64])
Set options: {'force_causal': False, 'force_causal_use_manual_mask': False, 'extend_context': False, 'first_frame_16': False, 'skip_vanilla_temporal_module': False, 'save_motion_module': False, 'auto_step_batch_offset': False, 'print_motion_module': False, 'print_model': False, 'offset_positional_encoding': False}


[(0, 0.004833221435546875),
 (1, 0.00525665283203125),
 (2, 0.007843017578125),
 (3, 0.011474609375),
 (4, 0.00963592529296875),
 (5, 0.006561279296875),
 (6, 0.00496673583984375),
 (7, 0.0101776123046875),
 (8, 0.98681640625),
 (9, 0.046630859375),
 (10, 0.0249481201171875),
 (11, 0.0179290771484375),
 (12, 0.006786346435546875),
 (13, 0.004302978515625),
 (14, 0.00522613525390625),
 (15, 0.00441741943359375),
 (16, 0.0),
 (17, 0.0),
 (18, 0.0),
 (19, 0.0),
 (20, 0.0),
 (21, 0.0),
 (22, 0.0),
 (23, 0.0),
 (24, 0.0),
 (25, 0.0),
 (26, 0.0),
 (27, 0.0),
 (28, 0.0),
 (29, 0.0),
 (30, 0.0),
 (31, 0.0)]

In [47]:
# When we throw on the causal mask, the gradient should be zero for future frames
get_frame_grads(8, True)

Set options: {'force_causal': True, 'force_causal_use_manual_mask': False, 'extend_context': False, 'first_frame_16': False, 'skip_vanilla_temporal_module': False, 'save_motion_module': False, 'auto_step_batch_offset': False, 'print_motion_module': False, 'print_model': False, 'offset_positional_encoding': False}


[(0, 0.017333984375),
 (1, 0.0175628662109375),
 (2, 0.0150909423828125),
 (3, 0.02435302734375),
 (4, 0.02313232421875),
 (5, 0.0156402587890625),
 (6, 0.0164031982421875),
 (7, 0.01525115966796875),
 (8, 0.98388671875),
 (9, 0.0),
 (10, 0.0),
 (11, 0.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0),
 (15, 0.0),
 (16, 0.0),
 (17, 0.0),
 (18, 0.0),
 (19, 0.0),
 (20, 0.0),
 (21, 0.0),
 (22, 0.0),
 (23, 0.0),
 (24, 0.0),
 (25, 0.0),
 (26, 0.0),
 (27, 0.0),
 (28, 0.0),
 (29, 0.0),
 (30, 0.0),
 (31, 0.0)]

In [48]:
# Another thing is that the data is channel major
# You can see the gradients do not flow across channels
get_frame_grads(16 + 8, True)

Set options: {'force_causal': True, 'force_causal_use_manual_mask': False, 'extend_context': False, 'first_frame_16': False, 'skip_vanilla_temporal_module': False, 'save_motion_module': False, 'auto_step_batch_offset': False, 'print_motion_module': False, 'print_model': False, 'offset_positional_encoding': False}


[(0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.0),
 (11, 0.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0),
 (15, 0.0),
 (16, 0.0169830322265625),
 (17, 0.01751708984375),
 (18, 0.01493072509765625),
 (19, 0.024139404296875),
 (20, 0.023193359375),
 (21, 0.01538848876953125),
 (22, 0.016204833984375),
 (23, 0.01464080810546875),
 (24, 0.9833984375),
 (25, 0.0),
 (26, 0.0),
 (27, 0.0),
 (28, 0.0),
 (29, 0.0),
 (30, 0.0),
 (31, 0.0)]